In [1]:
PROJECT_ID = "formazione-riccardo-zanella"
REGION = 'us-central1'
BUCKET_NAME = "bbs-2021-opml4b-explainability"

In [2]:
EXPORT_PATH = 'gs://' + BUCKET_NAME + '/mdl/image_data/image_data_20211103_112738/model/'
MODEL = 'flowers'

# Model load

In [3]:
import tensorflow as tf
import numpy as np

2021-11-18 15:18:38.347824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-18 15:18:38.347874: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
model = tf.keras.models.load_model(EXPORT_PATH)
# Print the names of your tensors
print('Model input tensor: ', model.input.name)
print('Model output tensor: ', model.output.name)

2021-11-18 15:18:43.648454: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-18 15:18:43.648505: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-18 15:18:43.648521: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (INJ-NB-126): /proc/driver/nvidia/version does not exist
2021-11-18 15:18:43.648714: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model input tensor:  conv2d_input
Model output tensor:  dense/Softmax:0


## Check for all needed signatures

In [5]:
input_name = list(model.signatures['xai_model'].structured_input_signature[1].keys())[0]
print(input_name)

output_name = list(model.signatures['xai_model'].structured_outputs.keys())[0]
print(output_name)

preprocess_name = list(model.signatures['xai_preprocess'].structured_input_signature[1].keys())[0]
print(preprocess_name)

numpy_inputs
output_0
bytes_inputs


In [6]:
from explainable_ai_sdk.metadata.tf.v2 import SavedModelMetadataBuilder
# We want to explain 'xai_model' signature.
builder = SavedModelMetadataBuilder(EXPORT_PATH, signature_name='xai_model')
random_baseline = np.random.rand(192, 192, 3)
builder.set_image_metadata(
    'numpy_inputs',
    input_baselines=[random_baseline.tolist()])
builder.save_metadata(EXPORT_PATH)

# Model deploy

In [7]:
! gcloud ai-platform models create $MODEL --enable-logging --region=$REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ai platform model [projects/formazione-riccardo-zanella/models/flowers].


In [8]:
VERSION = 'v1'

In [9]:
! gcloud beta ai-platform versions create $VERSION --region=$REGION \
--model $MODEL \
--origin $EXPORT_PATH \
--runtime-version 2.4 \
--framework TENSORFLOW \
--python-version 3.7 \
--machine-type n1-standard-4 \
--explanation-method xrai \
--num-integral-steps 50    

Using endpoint [https://us-central1-ml.googleapis.com/]
Explanations reflect patterns in your model, but don't necessarily reveal fundamental relationships about your data population. See https://cloud.google.com/vertex-ai/docs/explainable-ai/limitations for more information.
Creating version (this might take a few minutes)......done.


In [10]:
# Make sure the model deployed correctly. State should be `READY` in the following log
! gcloud ai-platform versions describe $VERSION --region $REGION --model $MODEL

Using endpoint [https://us-central1-ml.googleapis.com/]
createTime: '2021-11-18T14:26:19Z'
deploymentUri: gs://bbs-2021-opml4b-explainability/mdl/image_data/image_data_20211103_112738/model/
etag: TyYFbbpbwJY=
explanationConfig:
  xraiAttribution:
    numIntegralSteps: 50
framework: TENSORFLOW
isDefault: true
machineType: n1-standard-4
name: projects/formazione-riccardo-zanella/models/flowers/versions/v1
pythonVersion: '3.7'
runtimeVersion: '2.4'
state: READY
